## Open Data and Imports 

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import descartes 
from shapely.geometry import Point, Polygon
import statsmodels.api as sm
import pmdarima as pm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import  ARIMA
from sklearn.metrics import mean_squared_error
%matplotlib inline 

In [ ]:
raw_df = pd.read_csv('Seoul.csv')
df_measurements = pd.read_csv('Measurement_info.csv')
df_station = pd.read_csv('Measurement_station_info.csv')

In [ ]:
raw_df.head()

##### This is a general indiction of what levels of certain pollutants are the worse.

In [ ]:
df_measurements

##### Station based on district of Seoul

In [ ]:
df_station

##### Cutting down to look at the daily average and creating a df that looks at every station and date seperatley 

In [ ]:
raw_df['Measurement date'] = raw_df['Measurement date'].str.slice(0, 11)
df = raw_df.groupby(['Station code', 'Measurement date']).aggregate({'SO2': 'mean', 'NO2':'mean', 'O3':'mean', 'CO':'mean', 'PM10':'mean', 'PM2.5':'mean'})

# Functions Relevant for Creating New Columns

##### SO2 Level

In [ ]:
def SO2Level(x):
    if x <= 0.02:
        return "Good"
    elif 0.02 < x <= 0.05:
        return  "Normal"
    elif 0.05 < x <= 0.15:
        return "Bad"
    elif 0.15 < x <= 1.0:
        return  "Very Bad"
df['SO2 Level'] = df.apply(lambda row: SO2Level(row.SO2), axis=1)

##### NO2 Level

In [ ]:
def NO2Level(x):
    if x <= 0.03:
        return "Good"
    elif 0.03 < x <= 0.06:
        return "Normal"
    elif 0.06 < x <= 0.2:
        return "Bad"
    elif 0.2 < x <= 2.0:
        return "Very Bad"
df['NO2 Level'] = df.apply(lambda row: NO2Level(row.NO2), axis=1) 

##### CO Level

In [ ]:
def COLevel(x):
    if x <= 2:
        return "Good"
    elif 2 < x <= 9:
        return "Normal"
    elif 9 < x <= 15:
        return "Bad"
    elif 15 < x <= 50:
        return "Very Bad"
df['CO Level'] = df.apply(lambda row: COLevel(row.CO), axis=1)

##### O3 Level

In [ ]:
def O3Level(x):
    if x <= 0.03:
        return "Good"
    elif 0.03 < x <= 0.09:
        return "Normal"
    elif 0.09 < x <= 0.15:
        return "Bad"
    elif 0.15 < x <= 0.5:
        return "Very Bad"
df['O3 Level'] = df.apply(lambda row: O3Level(row.O3), axis=1)

##### PM10 Level

In [ ]:
def PM10Level(x):
    if x <= 30:
        return "Good"
    elif 30 < x <= 80:
        return "Normal"
    elif 80 < x <= 150:
        return "Bad"
    elif 150 < x <= 600:
        return "Very Bad"
df['PM10 Level'] = df.apply(lambda row: PM10Level(row.PM10), axis=1)

##### PM2.5 Level

In [ ]:
def PM25Level(x):
    if x <= 15:
        return "Good"
    elif 15 < x <= 35:
        return "Normal"
    elif 35 < x <= 75:
        return "Bad"
    elif 75 < x <= 500:
        return "Very Bad"
df['PM2.5 Level'] = df.apply(lambda row: PM25Level(row['PM2.5']), axis=1)

In [ ]:
df

# Some EDA 

In [ ]:
df.info()

Looking to see any correlations between any pollutants 

In [ ]:
df.corr().style.background_gradient(cmap='coolwarm')

Looking at Stations and different 

In [ ]:
#Pm2.5 Levels by Stations
fig, axes = plt.subplots(1, 1, figsize=(4, 6))
sns.boxplot(y='PM2.5', data=df, showfliers=False);

In [ ]:
#SO2 Levels by Stations
fig, axes = plt.subplots(1, 1, figsize=(4, 6))
sns.boxplot(y='SO2', data=df, showfliers=False);

In [ ]:
#NO2 Levels by Stations
fig, axes = plt.subplots(1, 1, figsize=(4, 6))
sns.boxplot(y='NO2', data=df, showfliers=False);

In [ ]:
#O3 Levels by Stations
fig, axes = plt.subplots(1, 1, figsize=(4, 6))
sns.boxplot(y='O3', data=df, showfliers=False);

In [ ]:
#PM10 Levels by Stations
fig, axes = plt.subplots(1, 1, figsize=(4, 6))
sns.boxplot(y='PM10', data=df, showfliers=False);

In [ ]:
df = df.reset_index()

In [ ]:
df['Measurement date'] = pd.to_datetime(df['Measurement date'])
df.set_index(df['Measurement date'], inplace=True)
df = df.drop('Measurement date', axis=1)
df

In [ ]:
df_station

Testing every station and see what pollutants are relavent to each station before modeling and testing. I will sperate every year and see if it shows the variation

In [ ]:
features = ['SO2', 'NO2', 'O3', 'CO', 'PM10', 'PM2.5']

## Relevant functions 

In [ ]:
def arima_results(endog, test, order=[0,0,0]):
    arima = ARIMA(endog, order=order).fit()
    
    train_hat = arima.predict()
    
    rmse_train = mean_squared_error(endog,
                                    train_hat,
                                    squared=False)
    
    
    rmse_test = mean_squared_error(test,
                                   arima.predict(), 
                                   squared=False)
    
 
    print(f"""Train RMSE: {rmse_train}""")     
    print(f"""Test RMSE: {rmse_test}""")    
    print(f"""Summary: {arima.summary()}""")
    
    return sm

In [ ]:
def stationarity_check(df):     
    df_test = adfuller(df)
    print('Results of Dickey-Fuller Test: \n')

    dfoutput = pd.Series(df_test[0:4], index=['Test Statistic', 'p-value', 
                                             '#Lags Used', 'Number of Observations Used'])
    for key,value in df_test[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    return None

In [ ]:
def arimamodel(timeseries):
    automodel = pm.auto_arima(timeseries, 
                              start_p=1, 
                              start_q=1,
                              test="adf",
                              seasonal=False,
                              trace=True)
    return automodel

In [ ]:
def plotarima(n_periods, timeseries, automodel):
    # Forecast
    fc, confint = automodel.predict(n_periods=n_periods, 
                                    return_conf_int=True)
    # Weekly index
    fc_ind = pd.date_range(timeseries.index[timeseries.shape[0]-1], 
                           periods=n_periods, 
                           freq="W")
    # Forecast series
    fc_series = pd.Series(fc, index=fc_ind)
    # Upper and lower confidence bounds
    lower_series = pd.Series(confint[:, 0], index=fc_ind)
    upper_series = pd.Series(confint[:, 1], index=fc_ind)
    # Create plot
    plt.figure(figsize=(10, 6))
    plt.plot(timeseries)
    plt.plot(fc_series, color="red")
    plt.xlabel("date")
    plt.ylabel(timeseries.name)
    plt.fill_between(lower_series.index, 
                     lower_series, 
                     upper_series, 
                     color="k", 
                     alpha=0.25)
    plt.legend(("past", "forecast", "95% confidence interval"),  
               loc="upper left")
    plt.show()

# Station 101

In [ ]:
df_101 = df[df['Station code'] == 101]

In [ ]:
df_101

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(50,10))
    fig.suptitle("Station 101", fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_101)

#### So as seen above for Station 101
- NO2 reaches levels of 'Bad' on 24 occasions and sits close to bad on most of the year. 
- PM10 reaches levels of 'Bad' on 46 occasions and 'Very Bad' on 4, can gets close to bad on many occasions. 
- Lastly, PM2.5 reaches levels of 'Bad' on 146 occasions and 'Very Bad' on 14 occasions. 
- Therefore, i will make a new df that only has those three pollutants since I am trying to test for pollutants that are deadly. 

In [ ]:
df_101

# Picking the relevant columns 

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_101 = df_101[columns]
df_101

# Train-Test Split

In [ ]:
train = df_101[:-542]
test = df_101[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_101['PM2.5']))
print(stationarity_check(df_101['PM10']))
print(stationarity_check(df_101['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# First Simple Model 

# Station 101 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_101['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_101['PM2.5']
automodel = arimamodel(df_101['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_101['PM2.5'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[2,0,2])

# Station 101 NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_101['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_101['NO2']
automodel = arimamodel(df_101['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_101['NO2'], trend='c', order=[1,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[1,0,2])

# Station 101 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_101['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_101['PM10']
automodel = arimamodel(df_101['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_101['PM10'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[2,0,2])

# Station 102

In [ ]:
df_102 = df[df['Station code'] == 102]

In [ ]:
df_102['PM2.5 Level'].value_counts()

In [ ]:
df_102.tail()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 102", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_102)

##### So as seen above for Station 102
- NO2 reaches levels of 'Bad' on 21 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 48 occasions and 'Very Bad' on 4, can gets close to bad on many occasions.
- Lastly, PM2.5 reaches levels of 'Bad' on 147 occasions and 'Very Bad' on 12 occasions.
- Therefore, i will make a new df that only has those three pollutants since I am trying to test for pollutants that are deadly.

# Picking the relevant columns

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_102 = df_102[columns]
df_102

# Train-Test Split

In [ ]:
train = df_102[:-542]
test = df_102[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_102['PM2.5']))
print(stationarity_check(df_102['PM10']))
print(stationarity_check(df_102['NO2']))

# Rolling Mean Difference Train¶

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test¶

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 102 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_102['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_102['PM10']
automodel = arimamodel(df_102['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_102['PM10'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[2,0,2])

# Station 102 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_102['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_102['PM2.5']
automodel = arimamodel(df_102['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_102['PM2.5'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[2,0,2])

# Station 102 NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_102['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_102['NO2']
automodel = arimamodel(df_102['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_102['NO2'], trend='c', order=[1,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[1,0,2])

# Station 103

In [ ]:
df_103 = df[df['Station code'] == 103]

In [ ]:
df_103['PM2.5 Level'].value_counts()

In [ ]:
df_103.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 103", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_103)

##### So as seen above for Station 103
- NO2 reaches levels of 'Bad' on 12 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 43 occasions and 'Very Bad' on 3, can gets close to bad on many occasions.
- Lastly, PM2.5 reaches levels of 'Bad' on 172 occasions and 'Very Bad' on 12 occasions.
- Therefore, i will make a new df that only has those three pollutants since I am trying to test for pollutants that are deadly.

# Picking the relevant columns

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_103 = df_103[columns]
df_103

In [ ]:
train = df_103[:-542]
test = df_103[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_103['PM2.5']))
print(stationarity_check(df_103['PM10']))
print(stationarity_check(df_103['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 103 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_103['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_103['PM10']
automodel = arimamodel(df_103['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_103['PM10'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[2,0,2])

# Station 103 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_103['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_103['PM2.5']
automodel = arimamodel(df_103['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_103['PM2.5'], trend='c', order=[1,0,3])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[2,0,2])


# Station 103 NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_103['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_103['NO2']
automodel = arimamodel(df_103['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_103['NO2'], trend='c', order=[4,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[4,0,2])

# Station 104

In [ ]:
df_104 = df[df['Station code'] == 104]

In [ ]:
df_104['PM2.5 Level'].value_counts()

In [ ]:
df_104.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 104", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_104)

##### So as seen above for Station 104
- NO2 reaches levels of 'Bad' on 11 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 69 occasions and 'Very Bad' on 5, can gets close to bad on many occasions.
- Lastly, PM2.5 reaches levels of 'Bad' on 183 occasions and 'Very Bad' on 19 occasions.
- Therefore, i will make a new df that only has those three pollutants since I am trying to test for pollutants that are deadly.

# Picking the relevant columns

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_104 = df_104[columns]
df_104

# Train-Test Split

In [ ]:
train = df_104[:-542]
test = df_104[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_101['PM2.5']))
print(stationarity_check(df_101['PM10']))
print(stationarity_check(df_101['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = train.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = test.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 104 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_104['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_104['PM10']
automodel = arimamodel(df_104['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_104['PM10'], trend='c', order=[2,0,1])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[2,0,1])

# Station 104 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_104['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_104['PM2.5']
automodel = arimamodel(df_104['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_104['PM2.5'], trend='c', order=[2,0,1])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[2,0,2])

# Station 104 PM NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_104['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_104['NO2']
automodel = arimamodel(df_104['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_104['NO2'], trend='c', order=[1,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[1,0,2])

# Station 105

In [ ]:
df_105 = df[df['Station code'] == 105]

In [ ]:
df_105['PM2.5 Level'].value_counts()

In [ ]:
df_105.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 105", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_105)

##### So as seen above for Station 105
- NO2 reaches levels of 'Bad' on 4 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 77 occasions and 'Very Bad' on 7, can gets close to bad on many occasions.
- Lastly, PM2.5 reaches levels of 'Bad' on 187 occasions and 'Very Bad' on 14 occasions.
- Therefore, i will make a new df that only has those three pollutants since I am trying to test for pollutants that are deadly.

# Picking the relevant columns

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_105 = df_105[columns]
df_105

# Train-Test Split

In [ ]:
train = df_105[:-542]
test = df_105[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_101['PM2.5']))
print(stationarity_check(df_101['PM10']))
print(stationarity_check(df_101['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 105 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_105['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_105['PM2.5']
automodel = arimamodel(df_105['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['PM2.5'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[2,0,2])

# Station 105 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_105['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_105['PM10']
automodel = arimamodel(df_105['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['PM10'], trend='c', order=[3,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[3,0,2])

# Station 105 NO2 

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_105['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_105['NO2']
automodel = arimamodel(df_105['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['NO2'], trend='c', order=[3,0,0])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[3,0,0])

# Station 106

In [ ]:
df_106 = df[df['Station code'] == 106]

In [ ]:
df_106.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 106", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_106)

In [ ]:
df_106['CO Level'].value_counts()

#### So as seen above for Station 106
- NO2 reaches levels of 'Bad' on 15 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 100 occasions and 'Very Bad' on 12 occasions, and gets close to 'Bad' on many occasions.
- PM2.5 reaches levels of 'Bad' on 250 occasions and 'Very Bad' on 60 occasions.


In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_106 = df_106[columns]
df_106

In [ ]:
train = df_106[:-542]
test = df_106[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_106['PM2.5']))
print(stationarity_check(df_106['PM10']))
print(stationarity_check(df_106['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 106 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_106['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_106['PM2.5']
automodel = arimamodel(df_106['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['PM2.5'], trend='c', order=[2,0,1])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[2,0,1])

# Station 106 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_106['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_106['PM10']
automodel = arimamodel(df_106['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['PM10'], trend='c', order=[3,0,1])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[3,0,1])

# Station 106 NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_106['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_106['NO2']
automodel = arimamodel(df_106['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_106['NO2'], trend='c', order=[3,0,0])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[3,0,0])

# Station 107

In [ ]:
df_107 = df[df['Station code'] == 107]

In [ ]:
df_107.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 107", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_107)

In [ ]:
df_107['NO2 Level'].value_counts()

###### So as seen above for Station 107
- NO2 reaches levels of 'Bad' on 9 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 114 occasions and 'Very Bad' on 12, can gets close to bad on many occasions.
- Lastly, PM2.5 reaches levels of 'Bad' on 198 occasions and 'Very Bad' on 24 occasions.

# Picking the relevant columns

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_107 = df_107[columns]
df_107

In [ ]:
train = df_107[:-542]
test = df_107[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_107['PM2.5']))
print(stationarity_check(df_107['PM10']))
print(stationarity_check(df_107['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));


In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 107 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_107['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_107['PM2.5']
automodel = arimamodel(df_107['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_107['PM2.5'], trend='c', order=[1,0,3])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[1,0,3])

# Station 107 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_107['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_107['PM10']
automodel = arimamodel(df_107['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_107['PM10'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[2,0,2])

# Station 107 NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_107['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_107['NO2']
automodel = arimamodel(df_107['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_107['NO2'], trend='c', order=[2,0,3])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[2,0,3])

# Station 108

In [ ]:
df_108 = df[df['Station code'] == 108]

In [ ]:
df_108.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 108", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_108)

In [ ]:
df_108['PM2.5 Level'].value_counts()

#### So as seen above for Station 108:
- NO2 reaches levels of 'Bad' on 15 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 93 occasions and 'Very Bad' on 6, can gets close to bad on many occasions.
- Lastly, PM2.5 reaches levels of 'Bad' on 255 occasions and 'Very Bad' on 52 occasions.

# Picking the relevant columns

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_108 = df_108[columns]
df_108

# Train-Test Split

In [ ]:
train = df_108[:-542]
test = df_108[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_108['PM2.5']))
print(stationarity_check(df_108['PM10']))
print(stationarity_check(df_108['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 108 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_108['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_108['PM2.5']
automodel = arimamodel(df_108['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['PM2.5'], trend='c', order=[2,0,1])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[2,0,2])

# Station 108 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_108['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_108['PM10']
automodel = arimamodel(df_108['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_108['PM10'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[2,0,2])

# Station 108 NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_108['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_108['NO2']
automodel = arimamodel(df_108['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['NO2'], trend='c', order=[1,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[1,0,2])

# Station 109

In [ ]:
df_109 = df[df['Station code'] == 109]

In [ ]:
df_109.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 109", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_109)

In [ ]:
df_109['PM2.5 Level'].value_counts()

#### So as seen above for Station 109
- NO2 reaches levels of 'Bad' on 37 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 60 occasions and 'Very Bad' on 5, can gets close to bad on many occasions.
- Lastly, PM2.5 reaches levels of 'Bad' on 154 occasions and 'Very Bad' on 19 occasions.


# Picking the relevant columns

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_109 = df_109[columns]
df_109

In [ ]:
train = df_109[:-542]
test = df_109[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_109['PM2.5']))
print(stationarity_check(df_109['PM10']))
print(stationarity_check(df_109['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 109 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_109['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_109['PM2.5']
automodel = arimamodel(df_109['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['PM2.5'], trend='c', order=[4,0,1])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[4,0,1])

# Station 109 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_109['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_109['PM10']
automodel = arimamodel(df_109['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_109['PM10'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[2,0,2])

# Station 109 NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_109['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_109['NO2']
automodel = arimamodel(df_109['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['NO2'], trend='c', order=[2,0,1])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[2,0,1])

# Station 110

In [ ]:
df_110 = df[df['Station code'] == 110]

In [ ]:
df_110.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 110", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_110)

In [ ]:
df_110['PM2.5 Level'].value_counts()

##### So as seen above for Station 110
- NO2 reaches levels of 'Bad' on 7 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 64 occasions and 'Very Bad' on 4, can gets close to bad on many occasions.
- Lastly, PM2.5 reaches levels of 'Bad' on 153 occasions and 'Very Bad' on 19 occasions.

# Picking the relevant columns

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_110 = df_110[columns]
df_110

In [ ]:
train = df_110[:-542]
test = df_110[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_110['PM2.5']))
print(stationarity_check(df_110['PM10']))
print(stationarity_check(df_110['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 110 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_110['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_110['PM2.5']
automodel = arimamodel(df_110['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_110['PM2.5'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[2,0,2])

# Station 110 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_110['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_110['PM10']
automodel = arimamodel(df_110['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_110['PM10'], trend='c', order=[3,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[3,0,2])

# Station 110 NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_110['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_110['NO2']
automodel = arimamodel(df_110['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_110['NO2'], trend='c', order=[5,0,4])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[5,0,4])

# Station 111

In [ ]:
df_111 = df[df['Station code'] == 111]

In [ ]:
df_111.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 111", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_111)

In [ ]:
df_111['PM2.5 Level'].value_counts()

###### So as seen above for Station 111
- NO2 reaches levels of 'Bad' on 26 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 78 occasions and 'Very Bad' on 11, can gets close to bad on many occasions.
- Lastly, PM2.5 reaches levels of 'Bad' on 166 occasions and 'Very Bad' on 17 occasions.

# Picking the relevant columns

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_111 = df_111[columns]
df_111

# Train-Test Split

In [ ]:
train = df_111[:-542]
test = df_111[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_111['PM2.5']))
print(stationarity_check(df_111['PM10']))
print(stationarity_check(df_111['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 111 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_111['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_111['PM2.5']
automodel = arimamodel(df_111['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_111['PM2.5'], trend='c', order=[4,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[4,0,2])

# Station 111 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_111['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_111['PM10']
automodel = arimamodel(df_111['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['PM10'], trend='c', order=[1,0,4])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[1,0,4])

# Station 111 NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_111['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_111['NO2']
automodel = arimamodel(df_111['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_105['NO2'], trend='c', order=[1,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[1,0,2])

# Station 112

In [ ]:
df_112 = df[df['Station code'] == 112]

In [ ]:
df_112.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 112", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_112)

In [ ]:
df_112['PM2.5 Level'].value_counts()

###### So as seen above for Station 112
- NO2 reaches levels of 'Bad' on 10 occasions and sits close to bad on most of the year.
- PM10 reaches levels of 'Bad' on 63 occasions and 'Very Bad' on 4, can gets close to bad on many occasions.
- PM2.5 reaches levels of 'Bad' on 144 occasions and 'Very Bad' on 14 occasions.

# Picking the relevant columns

In [ ]:
columns = ['NO2', 'PM10', 'PM2.5']
df_112 = df_112[columns]
df_112

# Train-Test Split

In [ ]:
train = df_112[:-542]
test = df_112[542:]

In [ ]:
test['NO2'].plot()

In [ ]:
test['PM10'].plot()

In [ ]:
test['PM2.5'].plot()

In [ ]:
train['NO2'].plot()

In [ ]:
train['PM10'].plot()

In [ ]:
train['PM2.5'].plot()

In [ ]:
print(stationarity_check(df_112['PM2.5']))
print(stationarity_check(df_112['PM10']))
print(stationarity_check(df_112['NO2']))

# Rolling Mean Difference Train

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_train = train.rolling(window=12).mean()
roll_mean_train.plot();

rm_diff_1 = roll_mean_train.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_train.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Rolling Mean Difference Test

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

roll_mean_test = test.rolling(window=12).mean()
roll_mean_test.plot();

rm_diff_1 = roll_mean_test.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean_test.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

In [ ]:
print(stationarity_check(rm_diff_1['PM2.5'].dropna()))
print(stationarity_check(rm_diff_1['PM10'].dropna()))
print(stationarity_check(rm_diff_1['NO2'].dropna()))

In [ ]:
print(stationarity_check(rm_diff_2['PM2.5'].dropna()))
print(stationarity_check(rm_diff_2['PM10'].dropna()))
print(stationarity_check(rm_diff_2['NO2'].dropna()))

# Station 112 PM2.5

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_112['PM2.5'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_112['PM2.5']
automodel = arimamodel(df_112['PM2.5'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_112['PM2.5'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM2.5'],test['PM2.5'], order=[2,0,2])

# Station 112 PM10

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_112['PM10'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_112['PM10']
automodel = arimamodel(df_112['PM10'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_112['PM10'], trend='c', order=[2,0,2])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['PM10'],test['PM10'], order=[2,0,2])

# Station 112 NO2

In [ ]:
base_model = sm.tsa.statespace.SARIMAX(df_112['NO2'], trend='c', order=[0,0,0])
res = base_model.fit(disp=False)
print(res.summary())

In [ ]:
timeseries = df_112['NO2']
automodel = arimamodel(df_112['NO2'])
plotarima(70, timeseries, automodel)

In [ ]:
best_model = sm.tsa.statespace.SARIMAX(df_112['NO2'], trend='c', order=[3,0,1])
res = best_model.fit(disp=False)
print(res.summary())

In [ ]:
model_train = arima_results(train['NO2'],test['NO2'], order=[3,0,1])

# Station 113

In [ ]:
df_113 = df[df['Station code'] == 113]

In [ ]:
df_113.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 113", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_113)

# Station 114

In [ ]:
df_114 = df[df['Station code'] == 114]

In [ ]:
df_114.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 114", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_114)

# Station 115

In [ ]:
df_115 = df[df['Station code'] == 115]

In [ ]:
df_115.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 115", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_115)

# Station 116

In [ ]:
df_116 = df[df['Station code'] == 116]

In [ ]:
df_116.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 116", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_116)

# Station 117

In [ ]:
df_117 = df[df['Station code'] == 117]

In [ ]:
df_117.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 117", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_117)

# Station 118

In [ ]:
df_118 = df[df['Station code'] == 118]

In [ ]:
df_118.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 118", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_118)

# Station 119

In [ ]:
df_119 = df[df['Station code'] == 119]

In [ ]:
df_119.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 119", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_119)

# Station 120

In [ ]:
df_120 = df[df['Station code'] == 120]

In [ ]:
df_120.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 120", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_120)

# Station 121

In [ ]:
df_121 = df[df['Station code'] == 121]

In [ ]:
df_121.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 121", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_121)

# Station 122

In [ ]:
df_122 = df[df['Station code'] == 122]

In [ ]:
df_122.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 122", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_122)

# Station 123

In [ ]:
df_123 = df[df['Station code'] == 123]

In [ ]:
df_123.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 123", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_123)

# Station 124

In [ ]:
df_124 = df[df['Station code'] == 124]

In [ ]:
df_124.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 124", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_124)

# Station 125

In [ ]:
df_125 = df[df['Station code'] == 125]

In [ ]:
df_125.head()

In [ ]:
for feature in features:
    fig, ax = plt.subplots(figsize=(35,8))
    fig.suptitle("Station 125", fontsize=20)
    ax.set_xlabel('Year, Day', fontsize=20)
    sns.lineplot(x='Measurement date', y=feature, data=df_125)

# Visualizations

In [ ]:
sns.countplot(x='NO2 Level',data=df, order=['Good', 'Normal', 'Bad'], color='red')

In [ ]:
sns.countplot(x='PM10 Level',data=df, order=['Good', 'Normal', 'Bad', 'Very Bad'], color='brown')

In [ ]:
sns.countplot(x='PM2.5 Level',data=df, order=['Good', 'Normal', 'Bad', 'Very Bad'], color='blue')